### Projeto final de Ciência dos Dados: Classificação de Pokémons a partir da sua cor e... 

##### Feito por:
Enrico Aloisi Nardi e Evandro Fontana Romeiro


##### Ideia geral do projeto:
A partir do que será apresentado, buscaremos tentar encontrar falhas no sistema de classificação da Nintendo partindo da premissa que Pokémons têm suas categorias definidas a partir da sua cor.

In [1]:
# Imports necessários para o projeto:
import os
import colorsys
import pandas as pd
from colorthief import ColorThief

In [55]:
lis_cores = [] #lista que guardará os códigos RGB das cores predominantes em cada foto

path = 'C:/Users/Enrico Aloisi Nardi/Desktop/Insper/SEGUNDO semestre/Ciência dos Dados/Projeto 3/Projeto-3--CDD/pokemon/'
arq_na_pasta= os.listdir(path)

for foto in arq_na_pasta:
    print(i)
    color_thief = ColorThief(path + foto)

    #recebendo a cor predominante de cada foto
    cor_predominante = color_thief.get_color(quality=1)

#     # Colocando a variável "cor predominante" numa função que converte um código de cores RGB em HSV e vice-versa.
#     # Ela será utilzada para aumentar a 'precisão' da leitura da cor da foto feita pelo ColouThief, que entrega o resultado em RGB.
#     # Com a conversão RGB-HSV buscamos reduzir confusões feitas pelo programa por conta de pequenas diferenças na cor lida.
#     cor_predominante_hsv =colorsys.rgb_to_hsv(cor_predominante[0], cor_predominante[1], cor_predominante[2])

    lis_cores.append(cor_predominante)

15
15
15
15
15


KeyboardInterrupt: 

In [ ]:
data= pd.read_csv('Pokemon.csv')

##### Fontes:
CSV com dados sobre os Pokémons: https://www.kaggle.com/abcsds/pokemon

Database de fotos: https://www.kaggle.com/kvpratama/pokemon-images-dataset